In [ ]:
# install reverse-geocoder https://pypi.org/project/reverse_geocoder/
!pip install reverse-geocoder

In [ ]:
import pandas as pd
import numpy as np
import reverse_geocoder as rg

In [ ]:
power_2019 = pd.read_excel('../power_data/2019_Annual_Summary.xls', skiprows=1)

WARNING *** file size (96452) not 512 + multiple of sector size (512)


In [ ]:
power_2019.head()

,Month,Date Event Began,Time Event Began,Date of Restoration,Time of Restoration,Area Affected,NERC Region,Alert Criteria,Event Type,Demand Loss (MW),Number of Customers Affected
0,January,01/05/2019,13:19:00,01/05/2019,15:07:00,Washington:,WECC,Complete loss of Interpersonal Communication a...,System Operations,0,Unknown
1,January,01/06/2019,03:00:00,01/09/2019,07:00:00,Washington:,WECC,"Loss of electric service to more than 50,000 c...",Severe Weather,230,230000
2,January,01/06/2019,17:56:00,01/06/2019,21:52:00,California: Sacramento County;,WECC,"Loss of electric service to more than 50,000 c...",Severe Weather,300,90382
3,January,01/06/2019,01:00:00,01/06/2019,12:00:00,"Washington: King County, Thurston County, Pier...",WECC,"Loss of electric service to more than 50,000 c...",Severe Weather,Unknown,230000
4,January,01/07/2019,20:57:00,01/07/2019,21:32:00,Michigan:,RF,Complete loss of monitoring or control capabil...,System Operations,0,0


In [ ]:
lightning_2019 = pd.read_csv('../weather_data/tvs-tiles-2019.csv', skiprows=2)

In [ ]:
lightning_2019.head()

,#ZDAY,CENTERLON,CENTERLAT,TOTAL_COUNT
0,20190101,-97.6,27.0,1
1,20190101,-152.3,60.9,3
2,20190101,-73.2,43.0,1
3,20190101,-135.7,57.0,1
4,20190101,-86.2,34.9,1


## Reverse Geocoding

Reverse geocoding is getting address (or county, state) from lat-lon.

We use the [reverse_geocoder](https://github.com/thampiman/reverse-geocoder) library
 It gives us county (admin 2) and state (admin 1) info too!


In [ ]:
# first add a new column containing both lat and lon
locations = list(zip(lightning_2019['CENTERLAT'],lightning_2019['CENTERLON']))
lightning_2019['location'] = locations

# then get the county (admin2) into a new column
address = rg.search(locations)
lightning_2019['county'] = [x['admin2'] for x in address]
lightning_2019['state'] = [x['admin1'] for x in address]

Loading formatted geocoded file...


In [ ]:
lightning_2019.head()

,#ZDAY,CENTERLON,CENTERLAT,TOTAL_COUNT,location,county,state
0,20190101,-97.6,27.0,1,"(27.0, -97.6)",Kenedy County,Texas
1,20190101,-152.3,60.9,3,"(60.9, -152.3)",Kenai Peninsula Borough,Alaska
2,20190101,-73.2,43.0,1,"(43.0, -73.2)",Bennington County,Vermont
3,20190101,-135.7,57.0,1,"(57.0, -135.7)",Sitka City and Borough,Alaska
4,20190101,-86.2,34.9,1,"(34.9, -86.2)",Madison County,Alabama


## Filtering only Severe Weather

In [ ]:
sev_weather_power_2019 = power_2019[power_2019['Event Type'].str.contains(r'Severe Weather', regex=True)]

In [ ]:
sev_weather_power_2019.head()

,Month,Date Event Began,Time Event Began,Date of Restoration,Time of Restoration,Area Affected,NERC Region,Alert Criteria,Event Type,Demand Loss (MW),Number of Customers Affected
1,January,01/06/2019,03:00:00,01/09/2019,07:00:00,Washington:,WECC,"Loss of electric service to more than 50,000 c...",Severe Weather,230,230000
2,January,01/06/2019,17:56:00,01/06/2019,21:52:00,California: Sacramento County;,WECC,"Loss of electric service to more than 50,000 c...",Severe Weather,300,90382
3,January,01/06/2019,01:00:00,01/06/2019,12:00:00,"Washington: King County, Thurston County, Pier...",WECC,"Loss of electric service to more than 50,000 c...",Severe Weather,Unknown,230000
8,January,01/12/2019,11:30:00,01/13/2019,22:00:00,Missouri: Jackson County; Kansas: Johnson County;,SPP RE,"Loss of electric service to more than 50,000 c...",Severe Weather,Unknown,112530
9,January,01/12/2019,11:30:00,Unknown,Unknown,Missouri: Nebraska:,SERC,"Loss of electric service to more than 50,000 c...",Severe Weather,Unknown,116600
